In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM,T5Tokenizer
from datasets import Dataset
import pandas as pd


2025-02-15 16:14:51.566546: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 16:14:51.588385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-15 16:14:51.600184: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-15 16:14:51.603170: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-15 16:14:51.617507: I tensorflow/core/platform/cpu_feature_guar

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

I0000 00:00:1739625294.206137   13383 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739625294.207879   13383 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739625294.207910   13383 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)


In [4]:
df=pd.read_csv('trip-data.csv')

In [5]:
df

,id,Activity,location,Description,Type,Price,openig-hours,holiday,act_time,time
0,0,Sharjah Archaeology Museum,Sharjah City,Ancient treasures and artifacts from UAE's pas...,museum,low,Saturday to Thursday:8:00 20:00 Friday:16:00 2...,0,8:00 20:00,"['morning','afternoon','evening']"
1,1,Bait Al Naboodah Museum,Sharjah City,"Historic Gulf home with coral walls, intricate...",museum,low,Saturday to Thursday:8:00 20:00 Friday:16:00 2...,0,8:00 20:00,"['morning','afternoon','evening']"
2,2,Bait Khalid Bin Ibrahim,Sharjah City,Historic house turned boutique hotel offers au...,old_hotel,low,Saturday to Thursday: 8:00 20:00 Friday: 16:00...,0,8:00 20:00,"['morning','afternoon','evening']"
3,3,Sharjah Calligraphy Museumc,Sharjah City,Arab world's only museum dedicated to calligra...,museum,low,Saturday to Thursday: 8:00 20:00 Friday: 16:00...,0,8:00 20:00,"['morning','afternoon','evening']"
4,4,Cultural Square,Sharjah City,Cultural Square: 14-metre high Quran sculpture...,museum,low,friday to friday:12:00 00:00,0,12:00 00:00,"['morning','afternoon','evening','night']"
...,...,...,...,...,...,...,...,...,...,...
1132,1182,Kalba Beach Corniche,East Coast,Explore the Kalba Beach Corniche in East Coast...,trendy,high,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"
1133,1183,Bait Al Naboodah Museum,Sharjah City,Explore the Bait Al Naboodah Museum in Sharjah...,precious,medium,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"
1134,1184,Najd Al Maqsar Village,Central Region,Explore the Najd Al Maqsar Village in Central ...,lively,medium,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"
1135,1185,Kalba Birds of Prey Centre,East Coast,Explore the Kalba Birds of Prey Centre in East...,bustling,medium,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"


In [6]:
df['input_text'] = df['location']+""+df['Type']
df['target_text'] = df['Description']

In [7]:
df['input_text'].str.len().max()

41

In [8]:
df['target_text'].str.len().max()

163

In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
train_dataset = Dataset.from_pandas(train_df[['input_text', 'target_text']])
test_dataset = Dataset.from_pandas(test_df[['input_text', 'target_text']])

In [11]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
I0000 00:00:1739625295.435700   13383 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739625295.435768   13383 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739625295.4

In [ ]:
tokenizer

In [13]:
tokenizer.pad_token_id=tokenizer.eos_token_id

In [14]:
from transformers import T5Tokenizer
import tensorflow as tf
def tokenize_function(examples):
    input_texts = [str(text) if isinstance(text, str) else "" for text in examples["input_text"]]
    target_texts = [str(text) if isinstance(text, str) else "" for text in examples["target_text"]]
    model_inputs = tokenizer(input_texts, padding="max_length", truncation=True, max_length=163, return_tensors="tf")
    labels = tokenizer(target_texts, padding="max_length", truncation=True, max_length=163, return_tensors="tf")
    decoder_input_ids = labels['input_ids'][:, :-1]
    decoder_input_ids = tf.concat([tf.zeros_like(decoder_input_ids[:, :1]), decoder_input_ids], axis=1)
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_input_ids"] = decoder_input_ids
    return model_inputs
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
def to_tf_dataset(dataset):
    return tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': dataset['input_ids'],
            'attention_mask': dataset['attention_mask'],
            'decoder_input_ids': dataset['decoder_input_ids'],
        },
        dataset['labels']
    ))
train_tf_dataset = to_tf_dataset(train_dataset).batch(8)
test_tf_dataset = to_tf_dataset(test_dataset).batch(8)


Map:   0%|          | 0/909 [00:00<?, ? examples/s]

Map:   0%|          | 0/228 [00:00<?, ? examples/s]

In [15]:
train_tf_dataset = to_tf_dataset(train_dataset).batch(1)
test_tf_dataset = to_tf_dataset(test_dataset).batch(1)


In [16]:
from keras.optimizers import Adam
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

In [17]:
model.fit(train_tf_dataset, validation_data=test_tf_dataset, epochs=15)


Epoch 1/15


I0000 00:00:1739625316.127512   13508 service.cc:146] XLA service 0x7fd1b540f6f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739625316.127572   13508 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-02-15 16:15:16.137414: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-15 16:15:16.153349: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
I0000 00:00:1739625316.192968   13508 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


909/909 [==============================] - 207s 201ms/step - loss: 0.1645 - accuracy: 0.9725 - val_loss: 0.1157 - val_accuracy: 0.9787
Epoch 2/15
909/909 [==============================] - 180s 198ms/step - loss: 0.1000 - accuracy: 0.9809 - val_loss: 0.1044 - val_accuracy: 0.9804
Epoch 3/15
909/909 [==============================] - 186s 205ms/step - loss: 0.0808 - accuracy: 0.9835 - val_loss: 0.1055 - val_accuracy: 0.9816
Epoch 4/15
909/909 [==============================] - 183s 202ms/step - loss: 0.0683 - accuracy: 0.9850 - val_loss: 0.1035 - val_accuracy: 0.9819
Epoch 5/15
909/909 [==============================] - 181s 199ms/step - loss: 0.0631 - accuracy: 0.9857 - val_loss: 0.1046 - val_accuracy: 0.9820
Epoch 6/15
909/909 [==============================] - 179s 197ms/step - loss: 0.0588 - accuracy: 0.9864 - val_loss: 0.1110 - val_accuracy: 0.9819
Epoch 7/15
909/909 [==============================] - 185s 204ms/step - loss: 0.0543 - accuracy: 0.9870 - val_loss: 0.1126 - val_accura

In [32]:
new_keywords = ["Sharjah City old_hotel"
]
input_ids = tokenizer(new_keywords, return_tensors="tf").input_ids
generated_ids = model.generate(input_ids, max_length=163, num_beams=5, no_repeat_ngram_size=2)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(generated_text)



Waterfront hotel and residences near city center with shuttle to beach.


In [21]:
tokenizer.pad_token_id

1

In [23]:
tokenizer.special_tokens_map

{'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '</s>',
 'additional_special_tokens': ['<extra_id_0>',
  '<extra_id_1>',
  '<extra_id_2>',
  '<extra_id_3>',
  '<extra_id_4>',
  '<extra_id_5>',
  '<extra_id_6>',
  '<extra_id_7>',
  '<extra_id_8>',
  '<extra_id_9>',
  '<extra_id_10>',
  '<extra_id_11>',
  '<extra_id_12>',
  '<extra_id_13>',
  '<extra_id_14>',
  '<extra_id_15>',
  '<extra_id_16>',
  '<extra_id_17>',
  '<extra_id_18>',
  '<extra_id_19>',
  '<extra_id_20>',
  '<extra_id_21>',
  '<extra_id_22>',
  '<extra_id_23>',
  '<extra_id_24>',
  '<extra_id_25>',
  '<extra_id_26>',
  '<extra_id_27>',
  '<extra_id_28>',
  '<extra_id_29>',
  '<extra_id_30>',
  '<extra_id_31>',
  '<extra_id_32>',
  '<extra_id_33>',
  '<extra_id_34>',
  '<extra_id_35>',
  '<extra_id_36>',
  '<extra_id_37>',
  '<extra_id_38>',
  '<extra_id_39>',
  '<extra_id_40>',
  '<extra_id_41>',
  '<extra_id_42>',
  '<extra_id_43>',
  '<extra_id_44>',
  '<extra_id_45>',
  '<extra_id_46>',
  '<extra_id_47>',
  

In [29]:
tokenizer.pad_token_id=0

In [30]:
tokenizer.eos_token_id=1

In [33]:
def generate_predictions(dataset, model, tokenizer):
    predictions, references = [], []
    for batch in dataset:
        input_ids = batch[0]['input_ids']
        attention_mask = batch[0]['attention_mask']
        labels = batch[1]
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=462)
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_refs = tokenizer.batch_decode(labels, skip_special_tokens=True)
        predictions.extend(decoded_preds)
        references.extend(decoded_refs)
    return predictions, references
predictions, references = generate_predictions(train_tf_dataset, model, tokenizer)

2025-02-15 18:14:23.759778: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [35]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [36]:
def calculate_bleu(predictions, references):
    bleu_scores = []
    for pred, ref in zip(predictions, references):
        ref_tokens = ref.split()
        pred_tokens = pred.split()
        score = sentence_bleu([ref_tokens], pred_tokens)
        bleu_scores.append(score)
    return sum(bleu_scores) / len(bleu_scores)
bleu_score = calculate_bleu(predictions, references)
print(f"Average BLEU Score: {bleu_score}")

Average BLEU Score: 0.5343529689622204


/home/mina/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/mina/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/mina/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunctio

In [37]:
def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1, rouge2, rougeL = 0, 0, 0
    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        rouge1 += scores['rouge1'].fmeasure
        rouge2 += scores['rouge2'].fmeasure
        rougeL += scores['rougeL'].fmeasure
    n = len(predictions)
    return {
        "rouge1": rouge1 / n,
        "rouge2": rouge2 / n,
        "rougeL": rougeL / n
    }
rouge_scores = calculate_rouge(predictions, references)
print(f"ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")

ROUGE-1: 0.7255
ROUGE-2: 0.6324
ROUGE-L: 0.7232


In [39]:
model.save_pretrained('T5BaseTripDescripton')

In [40]:
tokenizer.save_pretrained('T5BaseTripDescriptoontokenizer')

('T5BaseTripDescriptoontokenizer/tokenizer_config.json',
 'T5BaseTripDescriptoontokenizer/special_tokens_map.json',
 'T5BaseTripDescriptoontokenizer/spiece.model',
 'T5BaseTripDescriptoontokenizer/added_tokens.json')